## In this notebook we discover the best way to use an API to download data from Sentinel-2

API Overview:

https://scihub.copernicus.eu/userguide/APIsOverview

In this notebook we use OpenAPI: 
https://scihub.copernicus.eu/userguide/OpenSearchAPI

Further keyword doc for the API:
https://scihub.copernicus.eu/twiki/do/view/SciHubUserGuide/FullTextSearch?redirectedfrom=SciHubUserGuide.3FullTextSearch

Endpoint:
https://apihub.copernicus.eu/apihub/search

Possible Wrapper:
https://github.com/sentinelsat/sentinelsat

Wrapper Doc:
https://sentinelsat.readthedocs.io/en/stable/api_overview.html

In [1]:
from geopandas import GeoDataFrame

In [3]:
from dotenv import load_dotenv
import os
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [3]:
load_dotenv()
api_user = os.getenv("API_USER")
api_secret = os.getenv("API_SECRET")
api_url = os.getenv("API_URL")

In [1]:
api = SentinelAPI(api_user, api_secret, api_url)

In [4]:
# search by polygon, time, and SciHub query keywords
#footprint = geojson_to_wkt(read_geojson('global_pv_inventory_all/trn_polygons.geojson'))

In [20]:
footprint = 'POLYGON ((-112.8317392623121 33.29924007745022, -112.83171833156598 33.299240152532725, -112.83171608577777 33.298979015183214, -112.83173710635542 33.298978865017745, -112.8317392623121 33.29924007745022))'

In [2]:
test_polygon = 'POLYGON((49.753186143904465 9.791118103372561, 49.75370948463839 9.791895943992854, 49.75261080916379 9.79392905844176, 49.75203546759806 9.793365794544307, 49.753186143904465 9.791118103372561))'

In [26]:
# ! Hier noch den Download Pfad hinzufügen
# Und für nur ein product downloaden 
products = api.query(footprint,
                     date=('20220601', date(2023, 1, 11)),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30),
                     limit=1) #area_relation='IsWithin',#

In [27]:
# products is a OrderedDict, from which we can get ID and information's about the data
products

OrderedDict([('a36f98be-5cd3-4a48-9259-0253b1e802bd',
              {'title': 'S2A_MSIL2A_20230107T181731_N0509_R084_T12SUB_20230107T215055',
               'link': "https://apihub.copernicus.eu/apihub/odata/v1/Products('a36f98be-5cd3-4a48-9259-0253b1e802bd')/$value",
               'link_alternative': "https://apihub.copernicus.eu/apihub/odata/v1/Products('a36f98be-5cd3-4a48-9259-0253b1e802bd')/",
               'link_icon': "https://apihub.copernicus.eu/apihub/odata/v1/Products('a36f98be-5cd3-4a48-9259-0253b1e802bd')/Products('Quicklook')/$value",
               'summary': 'Date: 2023-01-07T18:17:31.024Z, Instrument: MSI, Satellite: Sentinel-2, Size: 1.12 GB',
               'ondemand': 'false',
               'generationdate': datetime.datetime(2023, 1, 7, 21, 50, 55),
               'beginposition': datetime.datetime(2023, 1, 7, 18, 17, 31, 24000),
               'endposition': datetime.datetime(2023, 1, 7, 18, 17, 31, 24000),
               'ingestiondate': datetime.datetime(2023,

In [28]:
# download all results from the search
api.download('a36f98be-5cd3-4a48-9259-0253b1e802bd', directory_path=r'C:\Users\Fabian\Documents\Masterarbeit_Daten')

MD5 checksumming:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

{'id': 'a36f98be-5cd3-4a48-9259-0253b1e802bd',
 'title': 'S2A_MSIL2A_20230107T181731_N0509_R084_T12SUB_20230107T215055',
 'size': 1205425353,
 'md5': '652b5eba6a0ecb6556d23ed278ceb478',
 'date': datetime.datetime(2023, 1, 7, 18, 17, 31, 24000),
 'footprint': 'POLYGON((-113.15103 33.42091875853871,-111.97035 33.435778798872015,-111.959595 32.44547285973725,-113.12721 32.431162035367194,-113.15103 33.42091875853871))',
 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('a36f98be-5cd3-4a48-9259-0253b1e802bd')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2023, 1, 7, 23, 43, 21, 481000),
 'Ingestion Date': datetime.datetime(2023, 1, 7, 23, 42, 13, 414000),
 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('a36f98be-5cd3-4a48-9259-0253b1e802bd')/Products('Quicklook')/$value",
 'path': 'C:\\Users\\Fabian\\Documents\\Masterarbeit_Daten\\S2A_MSIL2A_20230107T181731_N0509_R084_T12SUB_20230107T215055.zip',
 'downloaded_bytes': 1205425353}

### ChatGPT: Only download RGB and NIR Band

In [ ]:
from sentinelhub import SHConfig

# Set up your account credentials and configure the API
config = SHConfig()
config.sh_client_id = 'your_client_id'
config.sh_client_secret = 'your_client_secret'
config.instance_id = 'your_instance_id'

### First try to make the API call with requests

In [1]:
import requests

In [5]:
platformname = 'Sentinel-2'
beginposition = ['2023-01-01T00:00:00.000Z TO NOW']
footprint = 'Intersects(POLYGON((49.753186143904465 9.791118103372561, 49.75370948463839 9.791895943992854, 49.75261080916379 9.79392905844176, 49.75203546759806 9.793365794544307, 49.753186143904465 9.791118103372561)))'
cloudcoverpercentage = '[0 TO 20]'

In [6]:

payload = {
    'platformname': platformname, 
    'beginposition': beginposition,
    'footprint': footprint,
    'cloudcoverpercentage': cloudcoverpercentage
    }


In [7]:
# TODO path from endpoint (search trennen)
r = requests.get('https://apihub.copernicus.eu/apihub/search', params=payload)

In [8]:
r.url

'https://apihub.copernicus.eu/apihub/search?platformname=Sentinel-2&beginposition=2022-10-01T00%3A00%3A00.000Z+TO+NOW&footprint=Intersects%28POLYGON%28%2849.753186143904465+9.791118103372561%2C+49.75370948463839+9.791895943992854%2C+49.75261080916379+9.79392905844176%2C+49.75203546759806+9.793365794544307%2C+49.753186143904465+9.791118103372561%29%29%29&cloudcoverpercentage=%5B0+TO+20%5D'

In [2]:
# Set the API endpoint URL
url = 'https://scihub.copernicus.eu/apihub/odata/v1/Products'

# Set the search parameters
params = {
    'search': 'filename:S2* AND platformname:Sentinel-2 AND producttype:S2MSI1C',  # Search for Sentinel-2 L1C products
    '$select': 'Name,Size,ContentDate',  # Select the fields to return
    '$orderby': 'ContentDate desc',  # Sort the results by date (newest first)
    '$top': 1  # Limit the results to the newest product
}

# Set the authentication credentials
auth = ('taraman', 'y6HG%!6cF!86y$tK3')

# Send the request
response = requests.get(url, params=params, auth=auth)

# Parse the response JSON
result = response.json()

# Get the download URL for the product
download_url = result['value'][0]['Name']

# Download the product
response = requests.get(download_url, stream=True, auth=auth)

download_path = r'C:\Users\Fabian\Documents\Masterarbeit_Daten\test.zip'


# Write the downloaded data to a file
with open(download_path, 'wb') as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
# Set the API endpoint URL
url = 'https://scihub.copernicus.eu/apihub/odata/v1/Products'

# Set the search parameters
params = {
    'search': 'filename:S2* AND platformname:Sentinel-2 AND producttype:S2MSI1C',  # Search for Sentinel-2 L1C products
    '$select': 'Name,Size,ContentDate',  # Select the fields to return
    '$orderby': 'ContentDate desc',  # Sort the results by date (newest first)
    '$top': 1  # Limit the results to the newest product
}

# Set the authentication credentials
auth = ('taraman', 'y6HG%!6cF!86y$tK3')

# Send the request
response = requests.get(url, params=params, auth=auth)


In [5]:
response

<Response [401]>

In [4]:
# Parse the response JSON
result = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
root_url = 'https://scihub.copernicus.eu/apihub/odata/v1/Products'

### OpenSearch API

Docs: https://scihub.copernicus.eu/userguide/OpenSearchAPI

In [6]:
root_url = 'https://scihub.copernicus.eu/dhus/search?q='